In [ ]:
import matplotlib
import pandas as pd
import ipywidgets as widgets
import io
import plotly.express as px
from IPython.display import display
from ipywidgets import interact_manual, interactive,GridspecLayout, AppLayout, Button, Layout, Widget

In [ ]:
#retreive file with ANOVA data
result = pd.read_excel("EidenMicroArrayANOVAS.xlsx")
result.rename(columns = {'p-value(PKONS vs. PWTNS)':'p-value(PKO vs. PWT)'}, inplace=True)

In [ ]:
expressions = pd.read_excel("EditedMicroarraysExpValues.xlsx")

In [ ]:
##define GUI  
def choose_gene(change):
    if dropdown.value not in genes2:
        if dropdown.value != '':
            genes2.append(dropdown.value)
            with outgenes:
                display(dropdown.value)

In [ ]:
graph_out1 = widgets.Output()  
graph_out2 = widgets.Output()
    
def on_value_change(change):
    #just be here
    print('')
    
def on_go_button_clicked(go_button):
    mouse_means2, stats, sig_genes = get_vals(go_button)
#Mark errors
    if len(genes2) < 1:
        with outerrors:
            print("Choose at least one Gene")
    else:

       ###Graph expression   
        import plotly.graph_objects as go
        fig = px.bar(mouse_means2, x="Genotype", y="average",
                         facet_col = "Gene Symbol",
                         color = "Tissue",
                         barmode = 'group',
                     color_discrete_sequence= px.colors.sequential.Plasma,
                     error_y="error"
                  )
        for a in fig.layout.annotations:
            a.text = a.text.split("=")[1]
            fig.update_yaxes(title_text='')

        fig.show()

       ###Graph statistics         
        fig = px.bar(stats, x="Tissue", y="Fold-Change(PKO vs. PWT)",
                    width=800, height=400,
                    facet_col = "Gene Symbol",
                    color = "p-value(PKO vs. PWT)", range_color=[0,p_threshold.value] #Change so user input tells it what p and fc to use
              )
        #.for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
        fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))


        fig.show()

In [ ]:
style = {'description_width': 'initial'}
genes2 = []             
spacer = widgets.Label('')
genetitle = widgets.Label('Select Genes')

dropdown =  widgets.Combobox(
    options= ["sig_genes"],
    description='Gene',
    placeholder='Type here',
    layout={'width': '80%'})

   
outgenes = widgets.Output()   
outgraph = widgets.Output(layout={'border': '1px solid black'})  

        
gene_button = widgets.Button(description = 'Enter Gene', 
                button_style = '', 
                display='flex',
                flex_flow='column',
                align_items='stretch')
gene_button.on_click(choose_gene)

go_button = widgets.Button(description = 'Produce Output', 
                           button_style = '', 
                            display='flex',
                            flex_flow='column',
                            align_items='stretch')
go_button.on_click(on_go_button_clicked)

gene_button.observe(dropdown.value)
go_button.observe(dropdown.value)

p_threshold = widgets.BoundedFloatText(
    value=0.05,
    description='p-value:',
    disabled=False,
    width = '80%',
    min=0,
    max=0.1,
    step=0.01,
    style=style)
p_threshold.observe(on_value_change, names='value')

fc_threshold = widgets.BoundedFloatText(
    value=2,
    description='fold change (+/-):',
    disabled=False,
    min=1.5,
    max=20,
    step=0.5,
    width = '80%',
    style=style)
fc_threshold.observe(on_value_change, names='value')


graph_out1 = widgets.Output()  
graph_out2 = widgets.Output()  

In [ ]:
genesbox = widgets.VBox([genetitle,dropdown, gene_button])
#cellsbox = widgets.VBox([celllabel, Cells_drop])
genechoices = widgets.VBox([genesbox, outgenes,go_button])
outerrors = widgets.Output() 

graphit = widgets.VBox([graph_out1, graph_out2])
top_app = widgets.HBox([genechoices, outerrors]) ##Put seletcors for p and fc values here
app = widgets.VBox([top_app, graphit])

In [ ]:
def on_thresh_button_clicked(th_go_button):
    
    results_stringent = result[result["p-value(PKO vs. PWT)"] <= p_threshold.value]#Change so user input tells it what p and fc to use
    results_stringent2=results_stringent[(results_stringent['Fold-Change(PKO vs. PWT)']>= fc_threshold.value) | (results_stringent['Fold-Change(PKO vs. PWT)']<= -fc_threshold.value)]
    sig_genes0 = results_stringent2["Gene Symbol"].tolist()
    sig_genes = list(set(sig_genes0))
    mouse_means1 = expressions[expressions["Gene Symbol"].isin(genes2)]
    mouse_means2 = (mouse_means1.groupby(['Gene Symbol','Tissue','Genotype'])['Expression']
         .agg([('average','mean'),('error','sem')])
         .reset_index())
    stats = results_stringent2[results_stringent2["Gene Symbol"].isin(genes2)]
    
    #create next part of app
    dropdown.options = sig_genes
    display(app)
    return(mouse_means2, stats)

In [ ]:
threshold_title = widgets.Label('First, set p and fold-change')
th_go_button = widgets.Button(description = 'Set Thresholds', 
                           button_style = '', 
                            display='flex',
                            flex_flow='column',
                            align_items='stretch')
th_go_button.on_click(on_thresh_button_clicked)
#th_go_button.observe(dropdown.value)

In [ ]:
def get_vals(go_button):
    results_stringent = result[result["p-value(PKO vs. PWT)"] <= p_threshold.value]#Change so user input tells it what p and fc to use
    results_stringent2=results_stringent[(results_stringent['Fold-Change(PKO vs. PWT)']>= fc_threshold.value) | (results_stringent['Fold-Change(PKO vs. PWT)']<= fc_threshold.value)]
    sig_genes = results_stringent2["Gene Symbol"].tolist()
    mouse_means1 = expressions[expressions["Gene Symbol"].isin(genes2)]
    mouse_means2 = (mouse_means1.groupby(['Gene Symbol','Tissue','Genotype'])['Expression']
         .agg([('average','mean'),('error','sem')])
         .reset_index())
    stats = results_stringent2[results_stringent2["Gene Symbol"].isin(genes2)]
    return(mouse_means2, stats, sig_genes)

In [ ]:
thresholds = widgets.VBox([threshold_title,p_threshold, fc_threshold])
th_go = widgets.HBox([thresholds,th_go_button])
th_go